In [1]:
import os
import sys
from io import StringIO
import pandas as pd

In [3]:
hhresult_file = "/Users/michaeltisza/mike_tisza/sandbox/test_pyrgv1i/ct2_tmp/hhpred/AA_files/all_AA_seqs.no_phrogs.part_007.out.hhr"

In [12]:
with open(hhresult_file, 'r') as hrh:
    for line in hrh:
        if line.startswith('Query         '):
            gene_name = line.strip('Query         ').split(' ')[0]
            #print(gene_name)
        if line.startswith('>'):
            full_desc = line.rstrip('\n').strip('>')
            #print(full_desc.split(' ; '))
            full_stats = next(hrh, '').strip()
            #print(full_stats.split('  '))
            print(gene_name, full_desc.split(' ; ')[0], full_desc.split(' ; ')[2], full_stats.split('  ')[0])

test_pyrgv1i_2_1 PF00519.17 Papillomavirus helicase Probab=99.94
test_pyrgv1i_2_1 PF00799.20 Geminivirus Rep catalytic domain Probab=99.58
test_pyrgv1i_2_1 PF02407.16 Putative viral replication protein Probab=99.42
test_pyrgv1i_2_1 PF01057.17 Parvovirus non-structural protein NS1 Probab=97.96
test_pyrgv1i_2_1 PF05496.12 Holliday junction DNA helicase RuvB P-loop domain Probab=97.16
test_pyrgv1i_2_1 PF05272.11 Virulence-associated protein E Probab=97.11
test_pyrgv1i_2_1 PF00493.23 MCM P-loop domain Probab=97.06
test_pyrgv1i_2_1 PF00910.22 RNA helicase Probab=96.88
test_pyrgv1i_2_1 PF03969.16 AFG1-like ATPase Probab=96.76
test_pyrgv1i_2_1 PF14532.6 Sigma-54 interaction domain Probab=96.69
